## **Classificação de Imagens de Raio-X com Redes Neurais Convolucionais (CNNs)**

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torchmetrics

Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
# Importar bibliotecas necessárias
import os
import random
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import zipfile

# Definir sementes para reprodutibilidade
torch.manual_seed(101010)
np.random.seed(101010)
random.seed(101010)

In [3]:
# --- Criação do Conjunto de Validação ---

# Função para mover 50 arquivos aleatórios
def mover_arquivos(dir_classe_origem, dir_classe_destino, n=50):
    if not os.path.exists(dir_classe_destino):
        os.makedirs(dir_classe_destino)
    
    # Verifica se já não foi movido
    if len(os.listdir(dir_classe_destino)) > 0:
        print(f"Diretório de validação {dir_classe_destino} já contém arquivos. Pulando movimentação.")
        return

    arquivos = os.listdir(dir_classe_origem)
    # Garante que não tentará mover mais arquivos do que existem
    qtd_para_mover = min(n, len(arquivos))
    if qtd_para_mover < n:
        print(f"Aviso: Menos de {n} arquivos disponíveis em {dir_classe_origem}. Movendo {qtd_para_mover}.")
        
    arquivos_aleatorios = random.sample(arquivos, qtd_para_mover)
    for arquivo in arquivos_aleatorios:
        shutil.move(os.path.join(dir_classe_origem, arquivo), os.path.join(dir_classe_destino, arquivo))

# Descompactar os dados (se necessário)
if not os.path.exists('data/chestxrays'):
    print("Descompactando dados...")
    with zipfile.ZipFile('data/chestxrays.zip', 'r') as arquivo_zip:
        arquivo_zip.extractall('data')
else:
    print("Diretório de dados já existe.")

# Mover 50 imagens de cada classe para a pasta de validação
mover_arquivos('data/chestxrays/train/NORMAL', 'data/chestxrays/val/NORMAL')
mover_arquivos('data/chestxrays/train/PNEUMONIA', 'data/chestxrays/val/PNEUMONIA')

Diretório de dados já existe.
Diretório de validação data/chestxrays/val/NORMAL já contém arquivos. Pulando movimentação.
Diretório de validação data/chestxrays/val/PNEUMONIA já contém arquivos. Pulando movimentação.


In [4]:
# --- Definição das Transformações ---

media_transformacao = [0.485, 0.456, 0.406]
desvio_transformacao = [0.229, 0.224, 0.225]

# Transformação de TREINO com Data Augmentation
transformacao_treino = transforms.Compose([
    transforms.Resize((224, 224)), # Garante que todas as imagens tenham o tamanho esperado pelo ResNet
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=media_transformacao, std=desvio_transformacao)
])

# Transformação de VALIDAÇÃO e TESTE (sem augmentation)
transformacao_validacao_teste = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=media_transformacao, std=desvio_transformacao)
])

# --- Criação dos Datasets e DataLoaders ---

# Aplicar as transformações
dataset_treino = ImageFolder('data/chestxrays/train', transform=transformacao_treino)
dataset_validacao = ImageFolder('data/chestxrays/val', transform=transformacao_validacao_teste)
dataset_teste = ImageFolder('data/chestxrays/test', transform=transformacao_validacao_teste)

# Definir tamanho do lote
TAMANHO_LOTE = 32

# Criar os data loaders
carregador_treino = DataLoader(dataset_treino, batch_size=TAMANHO_LOTE, shuffle=True)
carregador_validacao = DataLoader(dataset_validacao, batch_size=TAMANHO_LOTE, shuffle=False)
carregador_teste = DataLoader(dataset_teste, batch_size=TAMANHO_LOTE, shuffle=False)

print(f"Dados de treino: {len(dataset_treino)} imagens")
print(f"Dados de validação: {len(dataset_validacao)} imagens")
print(f"Dados de teste: {len(dataset_teste)} imagens")

Dados de treino: 200 imagens
Dados de validação: 100 imagens
Dados de teste: 100 imagens


In [5]:
# --------------------------
# Instanciar o modelo
# --------------------------
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# --------------------------
# Modificar o modelo
# --------------------------
# Congelar todos os parâmetros
for parametro in resnet18.parameters():
    parametro.requires_grad = False

# Substituir a camada final (fully connected) para nossa classificação binária
resnet18.fc = nn.Linear(resnet18.fc.in_features, 1)

print("Modelo ResNet-18 modificado e pronto.")

Modelo ResNet-18 modificado e pronto.


In [6]:
from torchmetrics import Accuracy

# --------------------------------------------
# Definir o loop de treinamento
# --------------------------------------------

def treinar(modelo, carregador_treino, carregador_validacao, funcao_perda, otimizador, num_epocas):
    
    # Inicializar métrica de acurácia
    metrica_acuracia = Accuracy(task="binary")
    
    for epoca in range(num_epocas):
        # --- Fase de Treinamento ---
        modelo.train()
        perda_acumulada = 0.0
        acuracia_acumulada = 0.0

        for entradas, rotulos in carregador_treino:
            otimizador.zero_grad()
            
            rotulos = rotulos.float().unsqueeze(1) # Ajustar formato para BCEWithLogitsLoss
            saidas = modelo(entradas)
            perda = funcao_perda(saidas, rotulos)
            
            perda.backward()
            otimizador.step()

            perda_acumulada += perda.item() * entradas.size(0)
            # Calcular acurácia
            predicoes = torch.sigmoid(saidas)
            acuracia_acumulada += metrica_acuracia(predicoes, rotulos) * entradas.size(0)

        perda_treino = perda_acumulada / len(dataset_treino)
        acuracia_treino = acuracia_acumulada / len(dataset_treino)

        # --- Fase de Validação ---
        modelo.eval()
        perda_validacao_acumulada = 0.0
        acuracia_validacao_acumulada = 0.0
        
        with torch.no_grad():
            for entradas, rotulos in carregador_validacao:
                rotulos = rotulos.float().unsqueeze(1)
                saidas = modelo(entradas)
                perda = funcao_perda(saidas, rotulos)
                
                perda_validacao_acumulada += perda.item() * entradas.size(0)
                predicoes = torch.sigmoid(saidas)
                acuracia_validacao_acumulada += metrica_acuracia(predicoes, rotulos) * entradas.size(0)

        perda_validacao = perda_validacao_acumulada / len(dataset_validacao)
        acuracia_validacao = acuracia_validacao_acumulada / len(dataset_validacao)

        print(f'Época [{epoca+1}/{num_epocas}], '
              f'Perda Treino: {perda_treino:.4f}, Acurácia Treino: {acuracia_treino:.4f}, '
              f'Perda Val.: {perda_validacao:.4f}, Acurácia Val.: {acuracia_validacao:.4f}')

print("Função de treinamento 'treinar' definida.")

Função de treinamento 'treinar' definida.


In [7]:
# ----------------------------------------
# Fine-tune o modelo (HIPERPARÂMETROS MODIFICADOS)
# ----------------------------------------        
        
modelo = resnet18
NUM_EPOCAS = 20 # Aumentado de 3 para 20
TAXA_APRENDIZADO = 0.001 # Diminuído de 0.01 para 0.001

# Otimizador treinará apenas os parâmetros da nova camada 'fc'
otimizador = torch.optim.Adam(modelo.fc.parameters(), lr=TAXA_APRENDIZADO)

# Função de perda para classificação binária
funcao_perda = torch.nn.BCEWithLogitsLoss()

print("Iniciando o treinamento...")
treinar(modelo, carregador_treino, carregador_validacao, funcao_perda, otimizador, num_epocas=NUM_EPOCAS)
print("Treinamento concluído.")

Iniciando o treinamento...
Época [1/20], Perda Treino: 0.7609, Acurácia Treino: 0.4450, Perda Val.: 0.7208, Acurácia Val.: 0.4600
Época [2/20], Perda Treino: 0.6619, Acurácia Treino: 0.6300, Perda Val.: 0.7904, Acurácia Val.: 0.5000
Época [3/20], Perda Treino: 0.5880, Acurácia Treino: 0.7200, Perda Val.: 0.6614, Acurácia Val.: 0.6300
Época [4/20], Perda Treino: 0.5586, Acurácia Treino: 0.7850, Perda Val.: 0.6475, Acurácia Val.: 0.6600
Época [5/20], Perda Treino: 0.5124, Acurácia Treino: 0.7650, Perda Val.: 0.6967, Acurácia Val.: 0.5600
Época [6/20], Perda Treino: 0.4868, Acurácia Treino: 0.8250, Perda Val.: 0.5606, Acurácia Val.: 0.8100
Época [7/20], Perda Treino: 0.4447, Acurácia Treino: 0.8650, Perda Val.: 0.5187, Acurácia Val.: 0.8100
Época [8/20], Perda Treino: 0.4177, Acurácia Treino: 0.8600, Perda Val.: 0.4840, Acurácia Val.: 0.7800
Época [9/20], Perda Treino: 0.4066, Acurácia Treino: 0.8600, Perda Val.: 0.4443, Acurácia Val.: 0.8700
Época [10/20], Perda Treino: 0.4055, Acurácia 

In [8]:
# Importar as métricas corretas da tarefa
from torchmetrics import Precision, ConfusionMatrix

# -----------------------
# Código de Avaliação
# ----------------------- 

modelo.eval()

# Inicializar as métricas solicitadas
metrica_precisao = Precision(task="binary")
metrica_matriz_confusao = ConfusionMatrix(task="binary", num_classes=2)

todas_predicoes = []
todos_rotulos = []

with torch.no_grad():
  for entradas, rotulos in carregador_teste:
    saidas = modelo(entradas)
    
    # Sigmoid + round para obter predições binárias (0 ou 1)
    predicoes = torch.sigmoid(saidas).round() 

    todas_predicoes.extend(predicoes.cpu().tolist())
    todos_rotulos.extend(rotulos.cpu().unsqueeze(1).tolist())

# Converter listas para tensores DEPOIS que o loop terminar
todas_predicoes = torch.tensor(todas_predicoes)
todos_rotulos = torch.tensor(todos_rotulos)

# Calcular métricas
precisao_teste = metrica_precisao(todas_predicoes, todos_rotulos).item()
matriz_confusao_teste = metrica_matriz_confusao(todas_predicoes, todos_rotulos)
 
print(f"\n--- Avaliação Final no Conjunto de Teste ---")
print(f"Precisão (Precision): {precisao_teste:.4f}")
print(f"\nMatriz de Confusão:")
print(f"(Linhas = Real, Colunas = Predito)")
print(f"         [0: Normal] [1: Pneumonia]")
print(matriz_confusao_teste.numpy())


--- Avaliação Final no Conjunto de Teste ---
Precisão (Precision): 0.8444

Matriz de Confusão:
(Linhas = Real, Colunas = Predito)
         [0: Normal] [1: Pneumonia]
[[43  7]
 [12 38]]


**Quais desafios foram encontrados no treinamento?**

O modelo inicialmente não conseguia aprender, apresentando uma acurácia próxima de 50%. Isso foi causado pela falta de **aumento de dados (data augmentation)** e uma taxa de aprendizado muito alta (lr=0.01).

No novo treinamento, a acurácia de treino (91%) terminou mais alta que a acurácia de validação (84%), o que indica um leve **overfitting**.

A matriz de confusão mostrou 12 Falsos Negativos. Este é um desafio crítico, pois o modelo falhou em detectar a pneumonia em 12 casos. Reduzir esse número seria a prioridade.

**Como melhorar o desempenho do modelo?**

Atualmente, nós só treinamos a última camada (fc). A maior melhoria viria de "descongelar" algumas das camadas anteriores do ResNet e treiná-las com uma taxa de aprendizado muito pequena (ex: lr=0.00001). Isso permite que o modelo ajuste seus filtros pré-treinados especificamente para imagens de raio-X.

Também poderíamos testar outras arquiteturas. O ResNet-18 é leve e rápido. Um modelo maior, como um ResNet-50 ou EfficientNet, poderia capturar padrões mais complexos e oferecer maior acurácia, ao custo de mais tempo de treinamento.